In [1]:
import sys
import os
import pandas as pd
from pathlib import Path
from pympi.Elan import Eaf

os.chdir('/home/jetske22/sign_cas/Sign-Language-Thesis-main/')
%load_ext autoreload
%autoreload 2

In [2]:
# convert each video file to a 25 fps video file but stay the same speed
for filename in os.listdir('data/eaf_files/separated/'):
    if filename.split('.')[-1] != 'mp4':
        continue
    input_video = 'data/eaf_files/separated/'+filename
    output_video = 'data/eaf_files/separated/'+'conv_'+filename
    print(filename)

    # convert the video to 25 fps
    os.system(f'ffmpeg -y -i {input_video} -r 25 -loglevel quiet {output_video}')

    # overwrite the old file with the same name
    os.rename('data/eaf_files/separated/'+'conv_'+filename, 'data/eaf_files/separated/'+filename)

1176340_1a1.mp4
1176340_1b1.mp4
1176566_1a1.mp4
1176566_1b1.mp4
1176846_1a1.mp4
1176846_1b1.mp4
1177002_1a1.mp4
1177002_1b1.mp4
1177266_1a1.mp4
1177266_1b1.mp4
1177436_1a1.mp4
1177436_1b1.mp4
1177702_1a1.mp4
1177702_1b1.mp4
1177860_1a1.mp4
1177860_1b1.mp4
1177918_1a1.mp4
1177918_1b1.mp4
1178133_1a1.mp4
1178133_1b1.mp4
1178347_1a1.mp4
1178347_1b1.mp4
1178761_1a1.mp4
1178761_1b1.mp4
1178939_1a1.mp4
1178939_1b1.mp4
1179224_1a1.mp4
1179224_1b1.mp4
1179389_1a1.mp4
1179389_1b1.mp4
1179868_1a1.mp4
1179868_1b1.mp4
1180092_1a1.mp4
1180092_1b1.mp4
1180254_1a1.mp4
1180254_1b1.mp4
1180546_1a1.mp4
1180546_1b1.mp4
1180724_1a1.mp4
1180724_1b1.mp4
1181011_1a1.mp4
1181011_1b1.mp4
1181027_1a1.mp4
1181027_1b1.mp4
1181159_1a1.mp4
1181159_1b1.mp4
1181455_1a1.mp4
1181455_1b1.mp4
1181838_1a1.mp4
1181838_1b1.mp4
1182135_1a1.mp4
1182135_1b1.mp4
1182343_1a1.mp4
1182343_1b1.mp4
1182517_1a1.mp4
1182517_1b1.mp4
1182801_1a1.mp4
1182801_1b1.mp4
1183035_1a1.mp4
1183035_1b1.mp4
1183203_1a1.mp4
1183203_1b1.mp4
1183703_

In [3]:
def convert_speaker_code(code):
    """  Convert 'ber-XXX' to 'BXXX' format """
    parts = code.split('-')
    if len(parts) == 2:
        prefix = parts[0].upper()
        number = parts[1].zfill(3)
        return f"B{number}"
    else:
        raise ValueError("Invalid speaker code format")
    
def get_speaker_ids(eaf):
    """ Get the speaker IDs from the EAF file """
    tier_name_1, tier_name_2 = 'Gebärde_r_A', 'Gebärde_r_B'
    speaker_S1 = eaf.tiers[tier_name_1][2].get('PARTICIPANT')
    speaker_S2 = eaf.tiers[tier_name_2][2].get('PARTICIPANT')
    speaker_S1 = convert_speaker_code(speaker_S1)
    speaker_S2 = convert_speaker_code(speaker_S2)

    return speaker_S1, speaker_S2, tier_name_1, tier_name_2

def check_label_match(label, options):
    """ Check if the label contains any of the options """
    for option in options:
        if option in label:
            return True
    return False

def parse_tier(old_annotations):
    """ Parse the old annotations to the new format """
    options_shake = ['GEST-NM-SHAKE-HEAD', 'GEST-NM-KOPFSCHÜTTELN']
    options_nod = ['GEST-NM-NOD-HEAD', 'GEST-NM-KOPFNICKEN']
    annotations = []
    for annotation in old_annotations:
        label = annotation[2]
        start_ms = int(annotation[0])
        end_ms = int(annotation[1])
        if check_label_match(label, options_shake):
            annotations.append(Annotation('shake', start_ms, end_ms))
        if check_label_match(label, options_nod):
            annotations.append(Annotation('nod', start_ms, end_ms))
    return annotations

class Annotation:
    def __init__(self, label, start_ms, end_ms):
        assert start_ms < end_ms

        self.label = label
        self.start_ms = start_ms
        self.end_ms = end_ms

    def __repr__(self):
        return f'Annotation {self.label} at {self.start_ms}-{self.end_ms} ms'

def create_eafs(eaf_path, movements, output_folder, speaker, video_id, speaker_a_or_b, ms_range):
    """ Create EAF files for the movements """
    new_tier_name_movement = "Head movement"
    eaf = Eaf(eaf_path, 'pympi')
    new_media = []
    for media in eaf.get_linked_files():
        if "1"+speaker_a_or_b+"1" in media['MEDIA_URL']:
            new_media.append(media)
    # Each sentence
    for movement_i, movement in enumerate(movements):
        eaf_out_path = output_folder+"/"+video_id+"-"+str(movement_i)+"_"+speaker+".eaf"
        try:
            new_eaf = Eaf()
            new_eaf.add_tier(new_tier_name_movement)
            new_eaf.add_annotation(new_tier_name_movement, movement.start_ms+ms_range, movement.end_ms-movement.start_ms+ms_range, movement.label)
            new_media[0]['MEDIA_URL'] = str(output_folder+"/"+video_id+"-"+str(movement_i)+"_"+speaker+".mp4")
            new_eaf.media_descriptors = new_media
            new_eaf.to_file(eaf_out_path)
        except:
            print(f"Error creating eaf {eaf_out_path}")

def create_videos(video_in_path, movements, output_folder, speaker, video_id, ms_range):
    """ Create videos for the movements """
    for movement_i, movement in enumerate(movements):
        video_out_path = output_folder+"/"+video_id+"-"+str(movement_i)+"_"+speaker+".mp4"
        try:
            start_time_s = (movement.start_ms-ms_range)/1000
            duration_s = (movement.end_ms-movement.start_ms+(2*ms_range))/1000
            os.system(f'ffmpeg -y -i {video_in_path} -ss {start_time_s} -t {duration_s} -c:v libx264 -crf 18 -preset slow -f mp4 -loglevel quiet {video_out_path}')
        except:
            print(f"Error creating video {video_out_path}")
    return True

In [4]:
# 1 frame = 40ms
# 17 frames (35/2) = 680ms

def separate_speakers_eaf(input_folder, output_folder):
    """ Separate the speakers in the EAF files """
    
    for filename in os.listdir(input_folder):
        if filename.split('.')[-1] != 'eaf':
            continue
        eaf_path = input_folder+"/"+filename
        try:
            eaf = Eaf(eaf_path, 'pympi')
            speaker_1, speaker_2, tier_name_1, tier_name_2 = get_speaker_ids(eaf)
            tier_names = [tier_name_1, tier_name_2]
            abs = ['a', 'b']
            for speaker_i, speaker in enumerate([speaker_1, speaker_2]):
                movements = parse_tier(eaf.get_annotation_data_for_tier(tier_names[speaker_i]))
                if len(movements) == 0:
                    continue
                video_id = eaf_path.split('/')[-1].split('.')[0]
                create_eafs(eaf_path, movements, output_folder, speaker, video_id, abs[speaker_i], 680)
                video_in_path = input_folder+"/"+filename.split('.')[0] + "_1" + abs[speaker_i] + "1.mp4"
                create_videos(video_in_path, movements, output_folder, speaker, video_id, 680)
        except:
            print(f"Could not process {filename}")

In [6]:
separate_speakers_eaf('data/eaf_files/to_separate', 'data/eaf_files/separated')

Error creating eaf data/eaf_files/separated/1176340-0_B021.eaf
Error creating eaf data/eaf_files/separated/1176566-0_B025.eaf
Error creating eaf data/eaf_files/separated/1176566-1_B025.eaf
Error creating eaf data/eaf_files/separated/1176566-2_B025.eaf
Error creating eaf data/eaf_files/separated/1176566-3_B025.eaf
Error creating eaf data/eaf_files/separated/1176846-0_B011.eaf
Error creating eaf data/eaf_files/separated/1176846-1_B011.eaf
Error creating eaf data/eaf_files/separated/1176846-2_B011.eaf
Error creating eaf data/eaf_files/separated/1176846-3_B011.eaf
Error creating eaf data/eaf_files/separated/1176846-4_B011.eaf
Error creating eaf data/eaf_files/separated/1176846-0_B020.eaf
Error creating eaf data/eaf_files/separated/1176846-1_B020.eaf
Error creating eaf data/eaf_files/separated/1177002-0_B007.eaf
Error creating eaf data/eaf_files/separated/1177002-1_B007.eaf
Error creating eaf data/eaf_files/separated/1177002-2_B007.eaf
Error creating eaf data/eaf_files/separated/1177002-3_B